In [1]:
import math
from sklearn import datasets
import pandas as pd

In [2]:
#finds entropy
def Entropy(y):
    a=y[0].value_counts()
    den=y.shape[0]
    e=0
    for i in a:
        x=i/den
        log_term=math.log(x,10)
        e+=(-1)*x*log_term
    return e

In [3]:
#treenode class made for descison tree
class treeNode:
        def __init__(self,level,entropy,classes={},feature_entropy=[]):
            #about current level
            self.level=level
            #diiferent classes present in a node
            self.classes=classes
            #current entropy
            self.entropy=entropy
            #info that will be printed when we wanna print the node
            self.message=""
            if(len(feature_entropy)!=0):
                self.message="Splitting on feature "+str(feature_entropy[0])+  " with info gain " + str(feature_entropy[1])
            else:
                self.message="Reached leaf Node"
            self.kids=[]
            
        def printNode(self):
            print("Level  "+str(self.level))
            for i in self.classes:
                print("Count of "+str(i)+" = "+str(self.classes[i]))
            print("Current Entropy  is =  "+str(self.entropy))
            print(self.message)
            

            

In [4]:
def printTree(node):
    node.printNode()
    print()
    for i in node.kids:
        printTree(i)
    return

In [5]:
iris = datasets.load_iris()

In [6]:
df = pd.DataFrame(iris.data)
df.columns = ["sl", "sw", 'pl', 'pw']

In [7]:
#Function to find label for a value
#if MIN_Value <=val < (m + Mean_Value) / 2 then it is assigned label a
#if (m + Mean_Value) <=val < Mean_Value then it is assigned label b
#if (Mean_Value) <=val < (Mean_Value + MAX_Value)/2 then it is assigned label c
#if (Mean_Value + MAX_Value)/2 <=val <= MAX_Value  then it is assigned label d

def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

#Function to convert a continuous data into labelled data
#There are 4 lables  - a, b, c, d
def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df[old_feature_name].max()
    third = (maximum + second)/2
    return df[old_feature_name].apply(label, args= (first, second, third))

In [8]:
#Convert all columns to labelled data
df['sl_labeled'] = toLabel(df, 'sl')
df['sw_labeled'] = toLabel(df, 'sw')
df['pl_labeled'] = toLabel(df, 'pl')
df['pw_labeled'] = toLabel(df, 'pw')
df

,sl,sw,pl,pw,sl_labeled,sw_labeled,pl_labeled,pw_labeled
0,5.1,3.5,1.4,0.2,b,c,a,a
1,4.9,3.0,1.4,0.2,a,b,a,a
2,4.7,3.2,1.3,0.2,a,c,a,a
3,4.6,3.1,1.5,0.2,a,c,a,a
4,5.0,3.6,1.4,0.2,a,c,a,a
...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,c,b,c,d
146,6.3,2.5,5.0,1.9,c,a,c,d
147,6.5,3.0,5.2,2.0,c,b,c,d
148,6.2,3.4,5.4,2.3,c,c,d,d


In [9]:
df.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)

In [10]:
set(df['sl_labeled'])

{'a', 'b', 'c', 'd'}

In [11]:
def build_tree(df, y, unused_features,level=0):
    #base case
    # 1. unused is empty
    if len(unused_features)==0:
        return
    # 2. y contains only one distinct value
    if len(set(y[0]))==1:
        entropy=0
        classes=dict(y[0].value_counts())
        n=treeNode(level,entropy,classes)
        return n
    
    current_node_entropy=Entropy(y) 
   
    info_gain=0
    best_feature = ""
    #traversing over unused features and finding the best fit feature for splitting
    for f in unused_features:
        a=set(df[f])
        l=0
        
        #weighted sum of entropy required to find info gain 
        for i in a:
            y_=y[df[f]==i]
            multiplication_term=y_.shape[0]/y.shape[0]
            l+=multiplication_term*Entropy(y_)
            
        #info gain of feature f
        l=current_node_entropy-l
        
        #finding best_featue
        if(best_feature=="" or l>info_gain):
            best_feature=f
            info_gain=l

    
    #declaring rest of variable that are needed my decision tree node
    classes=dict(y[0].value_counts())
    feature_gain=[best_feature,info_gain]
    nodee=treeNode(level,current_node_entropy,classes,feature_gain)
   
    #dropping best_feature
    a=[]
    if(len(unused_features)>1):
         a=set(df[best_feature])
    unused_features=unused_features.drop([best_feature])
    
      
   
    #recursively making tree
    for i in a:
         nodee.kids.append(build_tree(df[df[best_feature]==i], y[df[best_feature]==i], unused_features,level+1))
    return nodee
    

In [12]:
y = pd.DataFrame(iris.target)
unused_features = df.columns.copy()
root=build_tree(df, y, unused_features)

In [13]:
printTree(root)

Level  0
Count of 2 = 50
Count of 1 = 50
Count of 0 = 50
Current Entropy  is =  0.4771212547196624
Splitting on feature pw_labeled with info gain 0.38011985380810287

Level  1
Count of 1 = 40
Count of 2 = 16
Current Entropy  is =  0.25982518101310587
Splitting on feature pl_labeled with info gain 0.09353304256650405

Level  2
Count of 1 = 39
Count of 2 = 8
Current Entropy  is =  0.19813531389382344
Splitting on feature sl_labeled with info gain 0.04753507701966517

Level  3
Count of 1 = 23
Count of 2 = 7
Current Entropy  is =  0.23594037110284793
Splitting on feature sw_labeled with info gain 0.02651450068167155

Level  3
Count of 1 = 2
Current Entropy  is =  0
Reached leaf Node

Level  3
Count of 1 = 14
Current Entropy  is =  0
Reached leaf Node

Level  3
Count of 2 = 1
Current Entropy  is =  0
Reached leaf Node

Level  2
Count of 2 = 8
Current Entropy  is =  0
Reached leaf Node

Level  2
Count of 1 = 1
Current Entropy  is =  0
Reached leaf Node

Level  1
Count of 2 = 34
Current Entro